In [41]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import torch

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim

class ShallowNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ShallowNet, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU() 
        self.fc2 = nn.Linear(hidden_size, output_size)  
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [43]:
df=pd.read_csv("data_daily.csv")
df['day_number'] = np.arange(len(df))
df['month'] = df['# Date'].str[5:7]
df['day'] = df['# Date'].str[8:].astype('int32')
df['lag_1'] = df['Receipt_Count'].shift(1)
df['lag_2'] = df['Receipt_Count'].shift(2)
df = df.dropna() 

In [45]:
X = df[['day_number', 'day', 'lag_1', 'lag_2']].values.astype('float32')
print(X)
y = df['Receipt_Count'].values.astype('float32')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

input_size = X_train.shape[1]
model = ShallowNet(input_size, 5, 1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 2000
for epoch in range(epochs):

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    mse = criterion(y_pred, y_test_tensor)
    print('Final Mean Squared Error on Test Data:', mse.item())
    rse = np.sqrt(mse.item())
    print(rse)


[[2.0000000e+00 3.0000000e+00 7.4555240e+06 7.5647660e+06]
 [3.0000000e+00 4.0000000e+00 7.0954140e+06 7.4555240e+06]
 [4.0000000e+00 5.0000000e+00 7.6661630e+06 7.0954140e+06]
 ...
 [3.6200000e+02 2.9000000e+01 1.0219445e+07 1.0350408e+07]
 [3.6300000e+02 3.0000000e+01 1.0313337e+07 1.0219445e+07]
 [3.6400000e+02 3.1000000e+01 1.0310644e+07 1.0313337e+07]]
Epoch [100/2000], Loss: 92416090112.0000
Epoch [200/2000], Loss: 73704374272.0000
Epoch [300/2000], Loss: 73703964672.0000
Epoch [400/2000], Loss: 73703505920.0000
Epoch [500/2000], Loss: 73703006208.0000
Epoch [600/2000], Loss: 73702440960.0000
Epoch [700/2000], Loss: 73701818368.0000
Epoch [800/2000], Loss: 73701171200.0000
Epoch [900/2000], Loss: 73700384768.0000
Epoch [1000/2000], Loss: 73699622912.0000
Epoch [1100/2000], Loss: 73698762752.0000
Epoch [1200/2000], Loss: 73697902592.0000
Epoch [1300/2000], Loss: 73696976896.0000
Epoch [1400/2000], Loss: 73696010240.0000
Epoch [1500/2000], Loss: 73694994432.0000
Epoch [1600/2000], 

In [40]:
torch.save(model.state_dict(), 'model.pth')